# CSV example

Obtain data from [https://www.ncei.noaa.gov/cdo-web/](https://www.ncei.noaa.gov/cdo-web/), for example the [daily summaries of the year 1949](https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1949.csv.gz).
Note that this is a gzipped file. Pandas can deal with it.

The data file has 94 MBytes, it may be too large to import into Excel (or Excel may cut some lines).

In [2]:
import pandas as pd
import requests

In [5]:
# download the file
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1949.csv.gz"
response = requests.get(url)
if response.ok:
    filename = url.rsplit('/', 1)[1]
    print(f"data downloaded. Will be saved as {filename}")
    with open(filename, "wb") as f:
        f.write(response.content)
else:
    print("An error occured while trying to retrieve the data from the internet.")

data downloaded. Will be saved as 1949.csv.gz


In [3]:
# download the file
url = "ftp://ftp.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1949.csv.gz"
response = requests.get(url)
if response.ok:
    filename = url.rsplit('/', 1)[1]
    print(f"data downloaded. Will be saved as {filename}")
    with open(filename, "wb") as f:
        f.write(response.content)
else:
    print("An error occured while trying to retrieve the data from the internet.")

InvalidSchema: No connection adapters were found for 'ftp://ftp.ncei.noaa.gov/pub/data/ghcn/daily/by_year/1949.csv.gz'

In [7]:
# read data into pandas
columns = ["stationcode", "datelabel", "param", "value", "mflag", "qflag", "sflag", "time"]
df = pd.read_csv(filename, names=columns, compression="gzip")
print(df)

          stationcode  datelabel param  value mflag qflag sflag  time
0         ACW00011604   19490101  TMAX    289   NaN   NaN     X   NaN
1         ACW00011604   19490101  TMIN    217   NaN   NaN     X   NaN
2         ACW00011604   19490101  PRCP      0   NaN   NaN     X   NaN
3         ACW00011604   19490101  SNOW      0   NaN   NaN     X   NaN
4         ACW00011604   19490101  SNWD      0   NaN   NaN     X   NaN
...               ...        ...   ...    ...   ...   ...   ...   ...
23787114  WZ004094600   19491231  PRCP      0   NaN   NaN     I   NaN
23787115  WZ004451000   19491231  PRCP      0   NaN   NaN     I   NaN
23787116  WZ004467410   19491231  PRCP    142   NaN   NaN     I   NaN
23787117  WZ004822290   19491231  PRCP      0   NaN   NaN     I   NaN
23787118  WZ004834260   19491231  PRCP    178   NaN   NaN     I   NaN

[23787119 rows x 8 columns]


In [8]:
dfg = df.groupby(["stationcode", "param"])
print(dfg)

In [9]:
dfg

In [15]:
print(dfg["value"].mean())

stationcode  param
ACW00011604  PGTM     868.000000
             PRCP      19.240000
             SNOW       0.000000
             SNWD       0.000000
             TMAX     285.250000
                         ...    
WZ004094600  PRCP      20.334247
WZ004451000  PRCP      27.797260
WZ004467410  PRCP      20.386301
WZ004822290  PRCP      36.613699
WZ004834260  PRCP      22.980822
Name: value, Length: 120876, dtype: float64
